In [1]:
import os
import nltk
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import logging
from sklearn.ensemble import RandomForestClassifier

In [11]:
# 读取sel日志，排序
sel_data = pd.read_csv('../dataset/raw_dataset/additional_sel_log_dataset.csv')
sel_data.sort_values(by=['sn', 'time'], inplace=True)
sel_data.reset_index(drop=True, inplace=True)
sel_data

,sn,time,msg
0,SERVER_30000,2020-10-13 03:06:01,Power Supply Power Unit | Fully Redundant
1,SERVER_30000,2020-10-13 03:06:11,Processor CPU 1 | Configuration Error | Asserted
2,SERVER_30001,2020-07-31 17:24:10,System ACPI Power State ACPI_PWR | S4/S5: sof...
3,SERVER_30001,2020-07-31 17:24:22,System ACPI Power State ACPI_PWR | S0/G0: wor...
4,SERVER_30001,2020-08-11 16:31:36,System ACPI Power State ACPI_PWR | S4/S5: sof...
...,...,...,...
732669,SERVER_42314,2019-11-15 13:37:34,Power Supply PS2_Status | Power Supply AC los...
732670,SERVER_42314,2019-11-15 13:37:36,Power Supply PS2_Status | Power Supply AC los...
732671,SERVER_42314,2020-02-03 16:06:16,Power Supply PS1_Status | Presence detected |...
732672,SERVER_42314,2020-02-03 16:06:16,Power Supply PS2_Status | Presence detected |...


In [13]:
test_data = sel_data.iloc[:100,:].copy()
test_data

,sn,time,msg
0,SERVER_30000,2020-10-13 03:06:01,Power Supply Power Unit | Fully Redundant
1,SERVER_30000,2020-10-13 03:06:11,Processor CPU 1 | Configuration Error | Asserted
2,SERVER_30001,2020-07-31 17:24:10,System ACPI Power State ACPI_PWR | S4/S5: sof...
3,SERVER_30001,2020-07-31 17:24:22,System ACPI Power State ACPI_PWR | S0/G0: wor...
4,SERVER_30001,2020-08-11 16:31:36,System ACPI Power State ACPI_PWR | S4/S5: sof...
...,...,...,...
95,SERVER_30001,2020-09-06 00:29:04,System Boot Initiated BIOS_Boot_Up | State As...
96,SERVER_30001,2020-09-06 00:36:36,System Boot Initiated BIOS_Boot_Up | State As...
97,SERVER_30002,2020-07-31 08:57:40,System ACPI Power State ACPI_PWR | S4/S5: sof...
98,SERVER_30002,2020-07-31 08:57:50,System ACPI Power State ACPI_PWR | S0/G0: wor...


In [14]:
test_data["msg"] = test_data["msg"].apply(lambda x: x.split("|"))
test_data["msg"] = test_data["msg"].apply(lambda x: sorted(x))
test_data

,sn,time,msg
0,SERVER_30000,2020-10-13 03:06:01,"[ Fully Redundant, Power Supply Power Unit ]"
1,SERVER_30000,2020-10-13 03:06:11,"[ Asserted, Configuration Error , Processor ..."
2,SERVER_30001,2020-07-31 17:24:10,"[ Asserted, S4/S5: soft-off , System ACPI Po..."
3,SERVER_30001,2020-07-31 17:24:22,"[ Asserted, S0/G0: working , System ACPI Pow..."
4,SERVER_30001,2020-08-11 16:31:36,"[ Asserted, S4/S5: soft-off , System ACPI Po..."
...,...,...,...
95,SERVER_30001,2020-09-06 00:29:04,"[ Asserted, State Asserted , System Boot Ini..."
96,SERVER_30001,2020-09-06 00:36:36,"[ Asserted, State Asserted , System Boot Ini..."
97,SERVER_30002,2020-07-31 08:57:40,"[ Asserted, S4/S5: soft-off , System ACPI Po..."
98,SERVER_30002,2020-07-31 08:57:50,"[ Asserted, S0/G0: working , System ACPI Pow..."


In [15]:
context = list(test_data["msg"].values)
context

[[' Fully Redundant', ' Power Supply Power Unit '],
 [' Asserted', ' Configuration Error ', ' Processor CPU 1 '],
 [' Asserted', ' S4/S5: soft-off ', ' System ACPI Power State ACPI_PWR '],
 [' Asserted', ' S0/G0: working ', ' System ACPI Power State ACPI_PWR '],
 [' Asserted', ' S4/S5: soft-off ', ' System ACPI Power State ACPI_PWR '],
 [' AC lost or out-of-range ', ' Asserted', ' Power Supply PSU0_Status '],
 [' Asserted', ' S4/S5: soft-off ', ' System ACPI Power State ACPI_PWR '],
 [' Asserted', ' S0/G0: working ', ' System ACPI Power State ACPI_PWR '],
 [' Asserted', ' S4/S5: soft-off ', ' System ACPI Power State ACPI_PWR '],
 [' Asserted', ' S0/G0: working ', ' System ACPI Power State ACPI_PWR '],
 [' Asserted', ' State Asserted ', ' System Boot Initiated BIOS_Boot_Up '],
 [' Asserted', ' S4/S5: soft-off ', ' System ACPI Power State ACPI_PWR '],
 [' Asserted', ' S0/G0: working ', ' System ACPI Power State ACPI_PWR '],
 [' Asserted', ' S4/S5: soft-off ', ' System ACPI Power State AC

In [17]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model = Word2Vec(sentences = context,workers=8, window=5, vector_size=10, epochs=30)
# model.save(model_save_path)

2022-03-10 22:40:23,294 : INFO : collecting all words and their counts
2022-03-10 22:40:23,295 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-03-10 22:40:23,296 : INFO : collected 18 word types from a corpus of 299 raw words and 100 sentences
2022-03-10 22:40:23,297 : INFO : Creating a fresh vocabulary
2022-03-10 22:40:23,297 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 9 unique words (50.0%% of original 18, drops 9)', 'datetime': '2022-03-10T22:40:23.297603', 'gensim': '4.1.2', 'python': '3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'prepare_vocab'}
2022-03-10 22:40:23,298 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 leaves 278 word corpus (92.97658862876254%% of original 299, drops 21)', 'datetime': '2022-03-10T22:40:23.298604', 'gensim': '4.1.2', 'python': '3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)]', 'pl